In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar
from core.s3 import S3AssetManager


In [2]:
notebook_name = "nutrinor_production_overview"
s3 = S3AssetManager(notebook_name=notebook_name)
PALETTE =  [ "#f9ee77","#1c8074","#1a494c", "#94af92", "#666666", "#f9ee77", "#f5ad68", "#c76931"]



In [3]:
df_pel1 = s3.read_excel("raw/nutrinor/Rendimiento pelletizadoras 2024 (3).xlsx",
                       sheet_name="rendimiento pellet 1 (420)",
                       skiprows=1
                       )

df_pel1["pellet"] = "pellet 420"
df_pel1.rename(columns={"OBSERVANCIONES": "OBSERVACIONES"}, inplace=True)

df_pel2 = s3.read_excel("raw/nutrinor/Rendimiento pelletizadoras 2024 (3).xlsx",
                       sheet_name="rendimiento pellet 2 (520)",
                       skiprows=1
                       )
df_pel2["pellet"] = "pellet 520"
df_pel2.rename(columns={"OBSERVANCIONES": "OBSERVACIONES"}, inplace=True)


df_pel3 = s3.read_excel("raw/nutrinor/Rendimiento pelletizadoras 2024 (3).xlsx",
                       sheet_name="rendimiento pellet 3 (350)",
                       skiprows=1
                       )
df_pel3["pellet"] = "pellet 350"
df_pel3.rename(columns={"OBSERVANCIONES": "OBSERVACIONES"}, inplace=True)

df_pel1.columns = [x.strip() for x in df_pel1.columns]
df_pel1["Fecha"] = pd.to_datetime(df_pel1["Fecha"])
df_pel1 = df_pel1[df_pel1["Fecha"].notnull()]


df_pel2.columns = [x.strip() for x in df_pel2.columns]
df_pel2["Fecha"] = pd.to_datetime(df_pel2["Fecha"])
df_pel2 = df_pel2[df_pel2["Fecha"].notnull()]


df_pel3.columns = [x.strip() for x in df_pel3.columns]
df_pel3["Fecha"] = pd.to_datetime(df_pel3["Fecha"])
df_pel3 = df_pel3[df_pel3["Fecha"].notnull()]

df = pd.concat([df_pel1, df_pel2, df_pel3])

cond_date_min = df["Fecha"]>='2025-01-01'
cond_date_max = df["Fecha"]<'2025-09-01'
df = df[cond_date_min & cond_date_max]

df_prod = df[df["Producto"]!='PAROS'].copy()

cond_obs = df_prod["OBSERVACIONES"].isnull()
df_prod["con_obs"] = np.where(cond_obs, "NO", "SI")

cols_num = [
    "toneladas",
    "tiempo trabajado (horas)",
    "ton/hora",
    "durabilidad (%)",
    'bultos',
    'carga (Hz)',

]
for col in cols_num:
    df_prod[col] = pd.to_numeric(df_prod[col], errors="coerce")
df_prod["toneladas/hora"] = df_prod["toneladas"]/df_prod["tiempo trabajado (horas)"]
df_prod = df_prod[df_prod["carga (Hz)"] < 30]

#TODO
cond1 = df_prod["toneladas/hora"]>1
cond2 = df_prod["toneladas/hora"]<12
df_prod_pel = df_prod[cond1 & cond2]

/Users/juandavidrincon/Documents/hawking/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/juandavidrincon/Documents/hawking/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/juandavidrincon/Documents/hawking/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
df_prod_pel.head()

,año,mes,semana,Fecha,Producto,Pelletizador,Hora inicial,Hora final,tiempo trabajado,tiempo trabajado (minutos),...,pellet,Prom/Dia/hora,Ton/sema,Horas/sema,Prom/sema,Ton/mesual,Horas/mensual,Prom/mensual,con_obs,toneladas/hora
1573,2025,1,2,2025-01-06,MAQUILA PREINICIADOR F1,Felipe,16:15:00,16:40:00,00:25:00,25,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,4.800000
1575,2025,1,2,2025-01-06,PREINICIADOR F0,Felipe,17:40:00,18:10:00,00:30:00,30,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4.000000
1576,2025,1,2,2025-01-06,CERDOS INICIACION 454,Wilson,18:30:00,20:00:00,01:30:00,90,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,5.333333
1577,2025,1,2,2025-01-06,CERDOS INICIACION,Wilson,20:20:00,21:50:00,01:30:00,90,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,5.333333
1595,2025,1,2,2025-01-09,DESARROLLO SUPERCERDO,Wilson-Daniel,04:50:00,06:50:00,02:00:00,120,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,5.000000


In [5]:
df_prod_pel["Producto"].unique()

array(['MAQUILA PREINICIADOR F1', 'PREINICIADOR F0',
       'CERDOS INICIACION 454', 'CERDOS INICIACION ',
       'DESARROLLO SUPERCERDO', 'INICIACION 12-25 HORIZONTES',
       'MARRANAS L', 'CERDOS', 'PERRO ADULTO', 'NUTRINOR FINALIZADOR',
       'MAQUILA PREINICIADOR EB', 'PREINICIADOR', 'GANADERIA PREPARACION',
       'SUPERLECHE GRANEL', 'ENGORDE HD', 'PRELEVANTE HD',
       'FINALIZADOR HD', 'LEVANTE HD', 'FINALIZADOR BR HORIZONTES',
       'LECHONES ', 'LEVANTE BR ', 'LEVANTE REEMPLAZOS SCP',
       'MAQUILA MARR-L EB ', 'ALTA ENERGIA GRANEL', 'PROBIOLECHE GRANEL',
       'ALTA ENERGIA CP GRANEL', 'POLLITO INICIACION',
       'LEVANTE BR HORIZONTES', 'FINALIZADOR HD 454', 'ENGORDE BR 454',
       'ALTA ENERGIA GV GRANEL', 'LECHE TOP GRANEL', 'LECHONES 454',
       'ALTA ENERGIA CP 454 GRANEL', 'PRODUCCION SALAS LB GRANEL',
       'POLLO ENGORDE ', 'MARRANAS GESTACION EB 454 SCP',
       'MARRANAS GESTACION EB', 'PROBIOLECHE', 'PRODUCCION SALAS LB',
       'DESEMPEÑO NORTE PELET',

In [6]:
df_prod_pel

,año,mes,semana,Fecha,Producto,Pelletizador,Hora inicial,Hora final,tiempo trabajado,tiempo trabajado (minutos),...,pellet,Prom/Dia/hora,Ton/sema,Horas/sema,Prom/sema,Ton/mesual,Horas/mensual,Prom/mensual,con_obs,toneladas/hora
1573,2025,1,2,2025-01-06,MAQUILA PREINICIADOR F1,Felipe,16:15:00,16:40:00,00:25:00,25,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,4.800000
1575,2025,1,2,2025-01-06,PREINICIADOR F0,Felipe,17:40:00,18:10:00,00:30:00,30,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4.000000
1576,2025,1,2,2025-01-06,CERDOS INICIACION 454,Wilson,18:30:00,20:00:00,01:30:00,90,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,5.333333
1577,2025,1,2,2025-01-06,CERDOS INICIACION,Wilson,20:20:00,21:50:00,01:30:00,90,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,5.333333
1595,2025,1,2,2025-01-09,DESARROLLO SUPERCERDO,Wilson-Daniel,04:50:00,06:50:00,02:00:00,120,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,2025,7,28,2025-07-12,TERNERAS ADVANCE,Jorge-Wilson,02:50:00,04:10:00,01:20:00,80,...,pellet 350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,3.000000
458,2025,7,28,2025-07-12,LEVANTE BR,Felipe-Sigifredo,11:45:00,21:45:00,10:00:00,600,...,pellet 350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,2.800000
459,2025,7,28,2025-07-12,LECHONES,Jorge-Wilson,22:10:00,1900-01-01 00:00:00,01:50:00,110,...,pellet 350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,3.207273
461,2025,7,29,2025-07-13,LECHONES,Jorge-Wilson,00:00:00,03:10:00,03:10:00,190,...,pellet 350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,3.195789


In [7]:
def summary(df, cols_group):
    df_group = df.groupby(cols_group).agg(
        toneladas_procesadas = ("toneladas", "sum"),
        horas_trabajadas = ("tiempo trabajado (horas)", "sum"),
        rendimiento_medio =("toneladas/hora", "median"),
        pdi_medio = ("durabilidad (%)", "median"),
        carga_media = ("carga (Hz)", "median"),
        bultos_procesado = ("bultos", "sum")
    ).reset_index()
    df_group["rendimiento_total"] = df_group["toneladas_procesadas"]/df_group["horas_trabajadas"]
    return df_group

In [8]:
summary(df=df_prod_pel, cols_group=["con_obs"])

,con_obs,toneladas_procesadas,horas_trabajadas,rendimiento_medio,pdi_medio,carga_media,bultos_procesado,rendimiento_total
0,NO,22765.44,4073.500000,6.000000,96.2,17.0,569136.0,5.588668
1,SI,845.12,175.916667,4.988571,96.0,20.0,21128.0,4.804093


In [9]:
summary(df=df_prod_pel, cols_group=["pellet"])

,pellet,toneladas_procesadas,horas_trabajadas,rendimiento_medio,pdi_medio,carga_media,bultos_procesado,rendimiento_total
0,pellet 350,2865.04,1003.916667,2.823529,96.0,16.0,71626.0,2.853862
1,pellet 420,8283.60,1548.583333,5.400000,96.4,26.0,207090.0,5.349147
2,pellet 520,12461.92,1696.916667,7.200000,96.2,16.0,311548.0,7.343861


In [10]:
summary_pel_date = summary(df=df_prod_pel, cols_group=["Fecha", "pellet"])

In [11]:

import plotly.express as px

fig = px.line(
    summary_pel_date,
    x="Fecha",
    y="rendimiento_medio",
    color="pellet",
    markers=True,
    color_discrete_map={
        "sin_obs": "#17877D",
        "con_obs": "#5A5A5A",
        "otro":    "#F6B27A",
    },
    title="Rendimiento Medio por Día"
)

fig.update_layout(
    xaxis_title="Fecha",
    yaxis_title="Rendimiento Medio (Ton/Hora)",
    plot_bgcolor="rgba(0,0,0,0)",
    paper_bgcolor="rgba(0,0,0,0)",
    font=dict(color="black"),
    xaxis=dict(tickangle=45)
)
fig.show()


In [12]:

def line_with_rolling(
    df,
    x: str,
    y: str,
    color: str,
    window="7D",                  # "7D" para 7 días (temporal) o 7 para 7 observaciones
    rolling_label="Media 7 días", # etiqueta de la curva de media
    rolling_color="#D62728",      # ROJO para destacar la media móvil
    show_markers=True,
    title="Serie diaria con media móvil",
    color_discrete_map=None       # opcional: dict de colores corporativos por categoría en 'color'
):
    """
    Dibuja la serie diaria (por categoría en 'color') y la media móvil por categoría en ROJO.
    - window: "7D", "14D", etc. para ventana temporal; o un entero para ventana por número de filas.
    """
    data = df.copy()

    # Asegurar fecha ordenable
    if not pd.api.types.is_datetime64_any_dtype(data[x]):
        data[x] = pd.to_datetime(data[x], errors="coerce")
    data = data.sort_values([color, x])

    # Calcular la media móvil por grupo
    def _rolling_group(g):
        g = g.set_index(x).sort_index()
        if isinstance(window, str):            # Ventana temporal: "7D"
            g["_rolling"] = g[y].rolling(window, min_periods=1).mean()
        else:                                  # Ventana por n observaciones
            g["_rolling"] = g[y].rolling(int(window), min_periods=1).mean()
        return g.reset_index()

    data_roll = data.groupby(color, group_keys=False).apply(_rolling_group)

    # Unir observado vs media móvil en formato largo (para line_dash)
    obs = data_roll[[x, color, y]].assign(serie="Observado").rename(columns={y: "valor"})
    mm  = data_roll[[x, color, "_rolling"]].assign(serie=rolling_label).rename(columns={"_rolling": "valor"})
    plot_df = pd.concat([obs, mm], ignore_index=True)

    # Construcción base con px (color por categoría, dash por tipo de serie)
    fig = px.line(
        plot_df,
        x=x, y="valor",
        color=color,
        line_dash="serie",
        markers=show_markers,
        title=title,
        color_discrete_map=(color_discrete_map or {})
    )

    # Ajustes de estilo: la media móvil SIEMPRE en ROJO y sin marcadores
    for tr in fig.data:
        # px nombra como "<categoria>, serie=Observado" | "<categoria>, serie=Media 7 días"
        if rolling_label in tr.name:
            tr.update(line=dict(color=rolling_color, width=3), mode="lines")  # sin markers
        else:
            tr.update(line=dict(width=2))  # observado mantiene color de la categoría

    # Layout pro: fondo transparente y marcos
    fig.update_layout(
        xaxis_title=x,
        yaxis_title=y,
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        font=dict(color="black"),
        xaxis=dict(tickangle=45, showline=True, linecolor="black", mirror=True),
        yaxis=dict(showline=True, linecolor="black", mirror=True, gridcolor="rgba(0,0,0,0.12)"),
        width=1200, height=500,
    )
    return fig


In [13]:
fig = line_with_rolling(
    df=summary_pel_date[summary_pel_date["pellet"] =='pellet 350'],
    x="Fecha",
    y="pdi_medio",     # <- tu métrica
    color="pellet",    # <- tu categoría (equivalente a hue)
    window="5D",       # "7D" = ventana temporal de 7 días; usa 7 si son 7 observaciones
    title="PDI medio por día con media móvil 7D",
    color_discrete_map={
        # opcional: colores corporativos por 'pellet'
        # "Pellet A": "#17877D",
        # "Pellet B": "#5A5A5A",
        # ...
    }
)
fig.show()


/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_97592/856353366.py:33: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [14]:
summary_pel_date["pellet"].unique()

array(['pellet 520', 'pellet 420', 'pellet 350'], dtype=object)

In [15]:


def facet_line_with_rolling(
    df: pd.DataFrame,
    pellets: list,                 # lista de pellets a graficar (orden del facet)
    metric: str,                   # columna numérica a graficar (p.ej. 'pdi_medio')
    date_col: str = "Fecha",
    pellet_col: str = "pellet",
    window="7D",                   # '7D' ventana temporal; o entero (p.ej. 7) para 7 observaciones
    title: str | None = None,
    ncols: int = 3,                # nº de columnas del facet (1xN)
    color_observado: str = "#17877D",
    color_mm: str = "#D62728",
    yaxis_title: str | None = None,
    show_markers: bool = True,
):
    """
    Genera un facet (1xN) con líneas Observado vs Media Móvil por pellet.
    - df: DataFrame con columnas [date_col, pellet_col, metric]
    - pellets: lista de valores de pellet a incluir y su orden en el facet
    - window: '7D' (temporal) o entero (n obs)
    - ncols: columnas del facet (si pellets > ncols, crea filas adicionales)
    """
    d = df.copy()
    d[date_col] = pd.to_datetime(d[date_col], errors="coerce")
    d = d[d[pellet_col].isin(pellets)].sort_values([pellet_col, date_col])

    # rolling por pellet
    def _roll(g):
        g = g.set_index(date_col).sort_index()
        if isinstance(window, str):
            g["mm"] = g[metric].rolling(window, min_periods=1).mean()
        else:
            g["mm"] = g[metric].rolling(int(window), min_periods=1).mean()
        return g.reset_index()

    rolled = d.groupby(pellet_col, group_keys=False).apply(_roll)

    # largo: Observado vs Media
    obs = rolled[[date_col, pellet_col, metric]].assign(serie="Observado").rename(columns={metric: "valor"})
    mm  = rolled[[date_col, pellet_col, "mm"]].assign(serie=f"Media {window}").rename(columns={"mm": "valor"})
    plot_df = pd.concat([obs, mm], ignore_index=True)

    # facet
    fig = px.line(
        plot_df,
        x=date_col, y="valor",
        facet_col=pellet_col, facet_col_wrap=ncols,
        color="serie",
        markers=show_markers,
        category_orders={pellet_col: pellets},
        title=title or f"{metric}: Observado vs Media {window}",
        color_discrete_map={
            "Observado": color_observado,
            f"Media {window}": color_mm
        }
    )

    # estilizar: media sin marcadores y más gruesa
    for tr in fig.data:
        if tr.name.endswith(f"Media {window}"):
            tr.update(mode="lines", line=dict(width=3, color=color_mm))
        else:
            tr.update(line=dict(width=2, color=color_observado))

    # layout: fondo transparente y marcos
    fig.update_layout(
        xaxis_title=date_col,
        yaxis_title=yaxis_title or metric,
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        font=dict(color="black")
    )
    for ax in [a for a in fig.layout if a.startswith(("xaxis","yaxis"))]:
        fig.layout[ax].update(showline=True, linecolor="black", mirror=True, gridcolor="rgba(0,0,0,0.12)")

    # si quieres igualar el rango Y entre facetas, descomenta:
    # fig.update_yaxes(matches='y')

    return fig


In [16]:
fig = facet_line_with_rolling(
    df=summary_pel_date,
    pellets=['pellet 520', 'pellet 420', 'pellet 350'],
    metric='pdi_medio',
    window='5D',
    title='PDI medio por día (Observado vs Media 5D)',
    ncols=1,  # 1x3
    yaxis_title='PDI medio (%)'
)
fig.show()
s3.save_plotly_html(fig, "pdi_by_date_pellet.html")

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_97592/4254109987.py:35: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [17]:
fig = facet_line_with_rolling(
    df=summary_pel_date,
    pellets=['pellet 520', 'pellet 420', 'pellet 350'],
    metric='rendimiento_medio',
    window='1D',
    title='Rendimiento medio por día (Observado vs Media 5D)',
    ncols=2,  # 1x3
    yaxis_title='Rend (Ton/Hora)'
)
fig.show()
s3.save_plotly_html(fig, "pdi_by_date_pellet.html")

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_97592/4254109987.py:35: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [18]:
fig = line_with_rolling(
    df=summary_pel_date,
    x="Fecha",
    y="rendimiento_medio",     # <- tu métrica
    color="pellet",    # <- tu categoría (equivalente a hue)
    window="7D",       # "7D" = ventana temporal de 7 días; usa 7 si son 7 observaciones
    title="Rendimiento promedio  por día con media móvil 7D",
    color_discrete_map={
        # opcional: colores corporativos por 'pellet'
        "pellet 520": "#17877D",
        "pellet 420": "#5A5A5A",
        "pellet 350": "#f5ad68"
        # ...'pellet 520', 'pellet 420', 'pellet 350'
    }
)
fig.show()
s3.save_plotly_html(fig, f"rendimiento_by_date_pellet.html")

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_97592/856353366.py:33: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [19]:
fig = line_with_rolling(
    df=summary_pel_date,
    x="Fecha",
    y="pdi_medio",     # <- tu métrica
    color="pellet",    # <- tu categoría (equivalente a hue)
    window="7D",       # "7D" = ventana temporal de 7 días; usa 7 si son 7 observaciones
    title="PDI promedio  por día con media móvil 7D",
    color_discrete_map={
        # opcional: colores corporativos por 'pellet'
        "pellet 520": "#17877D",
        "pellet 420": "#5A5A5A",
        "pellet 350": "#f5ad68"
        # ...'pellet 520', 'pellet 420', 'pellet 350'
    }
)
fig.show()
s3.save_plotly_html(fig, f"pdi_by_date_pellet.html")

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_97592/856353366.py:33: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [20]:
fig = line_with_rolling(
    df=summary_pel_date,
    x="Fecha",
    y="carga_media",     # <- tu métrica
    color="pellet",    # <- tu categoría (equivalente a hue)
    window="5D",       # "7D" = ventana temporal de 7 días; usa 7 si son 7 observaciones
    title="Carga media por día con media móvil 5D",
    color_discrete_map={
        # opcional: colores corporativos por 'pellet'
        # "Pellet A": "#17877D",
        # "Pellet B": "#5A5A5A",
        # ...
    }
)
fig.show()

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_97592/856353366.py:33: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [21]:
fig = line_with_rolling(
    df=summary_pel_date,
    x="Fecha",
    y="toneladas_procesadas",     # <- tu métrica
    color="pellet",    # <- tu categoría (equivalente a hue)
    window="5D",       # "7D" = ventana temporal de 7 días; usa 7 si son 7 observaciones
    title="Carga media por día con media móvil 5D",
    color_discrete_map={
        # opcional: colores corporativos por 'pellet'
        # "Pellet A": "#17877D",
        # "Pellet B": "#5A5A5A",
        # ...
    }
)
fig.show()

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_97592/856353366.py:33: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [22]:
summary_pel_date

,Fecha,pellet,toneladas_procesadas,horas_trabajadas,rendimiento_medio,pdi_medio,carga_media,bultos_procesado,rendimiento_total
0,2025-01-02,pellet 520,2.00,0.333333,6.000000,95.6,17.0,50.0,6.000000
1,2025-01-03,pellet 520,5.00,0.833333,6.000000,96.8,14.0,125.0,6.000000
2,2025-01-04,pellet 520,2.00,0.416667,4.800000,96.0,16.0,50.0,4.800000
3,2025-01-06,pellet 420,20.00,3.916667,5.066667,96.5,26.5,500.0,5.106383
4,2025-01-08,pellet 520,35.00,4.250000,8.000000,95.0,16.0,875.0,8.235294
...,...,...,...,...,...,...,...,...,...
352,2025-07-12,pellet 420,84.68,16.166667,5.818182,96.8,25.0,2117.0,5.237938
353,2025-07-12,pellet 520,101.20,14.166667,6.800000,96.0,15.0,2530.0,7.143529
354,2025-07-13,pellet 350,20.12,5.333333,3.905587,96.1,17.0,503.0,3.772500
355,2025-07-13,pellet 420,37.52,5.250000,6.703217,96.0,24.0,938.0,7.146667


In [23]:
# Peletizadora 420 en Junio reduron carga y rendimiento y calidad se ve afectados

In [24]:
summary_pel_date

,Fecha,pellet,toneladas_procesadas,horas_trabajadas,rendimiento_medio,pdi_medio,carga_media,bultos_procesado,rendimiento_total
0,2025-01-02,pellet 520,2.00,0.333333,6.000000,95.6,17.0,50.0,6.000000
1,2025-01-03,pellet 520,5.00,0.833333,6.000000,96.8,14.0,125.0,6.000000
2,2025-01-04,pellet 520,2.00,0.416667,4.800000,96.0,16.0,50.0,4.800000
3,2025-01-06,pellet 420,20.00,3.916667,5.066667,96.5,26.5,500.0,5.106383
4,2025-01-08,pellet 520,35.00,4.250000,8.000000,95.0,16.0,875.0,8.235294
...,...,...,...,...,...,...,...,...,...
352,2025-07-12,pellet 420,84.68,16.166667,5.818182,96.8,25.0,2117.0,5.237938
353,2025-07-12,pellet 520,101.20,14.166667,6.800000,96.0,15.0,2530.0,7.143529
354,2025-07-13,pellet 350,20.12,5.333333,3.905587,96.1,17.0,503.0,3.772500
355,2025-07-13,pellet 420,37.52,5.250000,6.703217,96.0,24.0,938.0,7.146667


In [25]:


def corr_map_plotly(
    df: pd.DataFrame,
    cols: list | None = None,
    method: str = "pearson",         # 'pearson', 'spearman', 'kendall'
    mask_upper: bool = True,         # enmascarar triángulo superior
    annotate: bool = True,           # mostrar valores sobre el mapa
    decimals: int = 2,               # redondeo de anotaciones
    colorscale: str = "RdBu",        # paleta (centrada en 0)
    title: str = "Mapa de correlación",
    sort_cols: str | None = None,    # None | 'variance' | 'alphabetical'
    bgcolor_transparent: bool = True # fondo transparente y marcos
):
    """
    Genera un mapa de correlación con Plotly (Heatmap).
    - df: DataFrame de entrada
    - cols: columnas a incluir (si None, usa numéricas)
    - method: 'pearson' | 'spearman' | 'kendall'
    - mask_upper: oculta triángulo superior (útil para no duplicar información)
    - annotate: escribe los coeficientes en el heatmap
    - decimals: decimales de las anotaciones
    - colorscale: escala de colores (p.ej., 'RdBu', 'Picnic', 'Viridis')
    - title: título de la figura
    - sort_cols: ordena columnas (None = tal cual, 'variance' por varianza descendente, 'alphabetical')
    - bgcolor_transparent: usa fondo transparente y marcos en ejes
    """
    # 1) Selección de columnas
    if cols is None:
        num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    else:
        num_cols = [c for c in cols if c in df.columns]
        # filtrar por numéricas dentro de 'cols'
        num_cols = [c for c in num_cols if pd.api.types.is_numeric_dtype(df[c])]
    if len(num_cols) < 2:
        raise ValueError("Se requieren al menos 2 columnas numéricas para calcular correlaciones.")

    data = df[num_cols].copy()

    # 2) Orden opcional de columnas para mejor lectura
    if sort_cols == "variance":
        order = data.var().sort_values(ascending=False).index.tolist()
        data = data[order]
    elif sort_cols == "alphabetical":
        data = data[sorted(data.columns)]

    # 3) Matriz de correlación
    corr = data.corr(method=method)

    # 4) Opcional: enmascarar triángulo superior (mantener diagonal)
    if mask_upper:
        corr_masked = corr.copy()
        corr_mask = np.triu(np.ones_like(corr_masked, dtype=bool), k=1)
        corr_masked = corr_masked.mask(corr_mask, other=np.nan)
        z = corr_masked.values
    else:
        z = corr.values

    xlabels = corr.columns.tolist()
    ylabels = corr.index.tolist()

    # 5) Heatmap
    fig = go.Figure(data=go.Heatmap(
        z=z,
        x=xlabels,
        y=ylabels,
        zmin=-1, zmax=1,                # rango fijo para correlaciones
        colorscale=colorscale,
        colorbar=dict(title="r", ticksuffix=""),
        hovertemplate="x=%{x}<br>y=%{y}<br>r=%{z:.2f}<extra></extra>",
        showscale=True
    ))

    # 6) Anotaciones (valores sobre cada celda)
    if annotate:
        annotations = []
        for i, yi in enumerate(ylabels):
            for j, xj in enumerate(xlabels):
                val = z[i][j]
                if np.isnan(val):
                    continue
                text = f"{val:.{decimals}f}"
                annotations.append(dict(
                    x=xj, y=yi, text=text,
                    xref="x1", yref="y1",
                    showarrow=False,
                    font=dict(color="black", size=11)
                ))
        fig.update_layout(annotations=annotations)

    # 7) Estilo pro (fondo y marcos)
    axis_common = dict(
        showline=True, linecolor="black", mirror=True,
        ticks="outside", ticklen=6,
        gridcolor="rgba(0,0,0,0.12)"
    )
    fig.update_xaxes(**axis_common)
    fig.update_yaxes(**axis_common, autorange="reversed")  # convención visual de matrices

    layout_updates = dict(
        title=dict(text=title, x=0.5),
        margin=dict(l=80, r=60, t=80, b=80),
        font=dict(family="Arial, Helvetica, sans-serif", size=12),
    )
    if bgcolor_transparent:
        layout_updates.update(
            plot_bgcolor="rgba(0,0,0,0)",
            paper_bgcolor="rgba(0,0,0,0)",
        )
    fig.update_layout(**layout_updates)
    return fig


In [26]:
summary_pel_date

,Fecha,pellet,toneladas_procesadas,horas_trabajadas,rendimiento_medio,pdi_medio,carga_media,bultos_procesado,rendimiento_total
0,2025-01-02,pellet 520,2.00,0.333333,6.000000,95.6,17.0,50.0,6.000000
1,2025-01-03,pellet 520,5.00,0.833333,6.000000,96.8,14.0,125.0,6.000000
2,2025-01-04,pellet 520,2.00,0.416667,4.800000,96.0,16.0,50.0,4.800000
3,2025-01-06,pellet 420,20.00,3.916667,5.066667,96.5,26.5,500.0,5.106383
4,2025-01-08,pellet 520,35.00,4.250000,8.000000,95.0,16.0,875.0,8.235294
...,...,...,...,...,...,...,...,...,...
352,2025-07-12,pellet 420,84.68,16.166667,5.818182,96.8,25.0,2117.0,5.237938
353,2025-07-12,pellet 520,101.20,14.166667,6.800000,96.0,15.0,2530.0,7.143529
354,2025-07-13,pellet 350,20.12,5.333333,3.905587,96.1,17.0,503.0,3.772500
355,2025-07-13,pellet 420,37.52,5.250000,6.703217,96.0,24.0,938.0,7.146667


In [27]:
for pel in summary_pel_date['pellet'].unique():
    summary_pel = summary_pel_date[summary_pel_date['pellet'] == pel]
    fig = corr_map_plotly(
        df=summary_pel,
        cols=["toneladas_procesadas", "horas_trabajadas", "rendimiento_medio", "pdi_medio", "carga_media"],
        method="pearson",
        mask_upper=True,
        annotate=True,
        decimals=2,
        colorscale="RdBu",
        title=f"Correlación (Pearson) — {pel}",
        sort_cols="variance"
    )
    fig.show()


In [28]:
operarios = [
    'Felipe-Wilson',
    'Felipe',
    'Luis',
    'Yohan-Luis',
    'Wilson',
    'Luis-Yohan',
    'Felipe-Sigifredo'

]

In [29]:
df_prod_pel

,año,mes,semana,Fecha,Producto,Pelletizador,Hora inicial,Hora final,tiempo trabajado,tiempo trabajado (minutos),...,pellet,Prom/Dia/hora,Ton/sema,Horas/sema,Prom/sema,Ton/mesual,Horas/mensual,Prom/mensual,con_obs,toneladas/hora
1573,2025,1,2,2025-01-06,MAQUILA PREINICIADOR F1,Felipe,16:15:00,16:40:00,00:25:00,25,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,4.800000
1575,2025,1,2,2025-01-06,PREINICIADOR F0,Felipe,17:40:00,18:10:00,00:30:00,30,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4.000000
1576,2025,1,2,2025-01-06,CERDOS INICIACION 454,Wilson,18:30:00,20:00:00,01:30:00,90,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,5.333333
1577,2025,1,2,2025-01-06,CERDOS INICIACION,Wilson,20:20:00,21:50:00,01:30:00,90,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,5.333333
1595,2025,1,2,2025-01-09,DESARROLLO SUPERCERDO,Wilson-Daniel,04:50:00,06:50:00,02:00:00,120,...,pellet 420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,2025,7,28,2025-07-12,TERNERAS ADVANCE,Jorge-Wilson,02:50:00,04:10:00,01:20:00,80,...,pellet 350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,3.000000
458,2025,7,28,2025-07-12,LEVANTE BR,Felipe-Sigifredo,11:45:00,21:45:00,10:00:00,600,...,pellet 350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,2.800000
459,2025,7,28,2025-07-12,LECHONES,Jorge-Wilson,22:10:00,1900-01-01 00:00:00,01:50:00,110,...,pellet 350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,3.207273
461,2025,7,29,2025-07-13,LECHONES,Jorge-Wilson,00:00:00,03:10:00,03:10:00,190,...,pellet 350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,3.195789


In [30]:

def cat_heatmap(
    df: pd.DataFrame,
    x_col: str,           # columna categórica para el eje X (ej: "Dado")
    y_col: str,           # columna categórica para el eje Y (ej: "pellet")
    value_col: str,       # columna numérica a pintar (ej: "rendimiento")
    aggfunc="mean",       # "mean", "median", "sum", etc.
    x_order: list | None = None,  # orden opcional de categorías en X
    y_order: list | None = None,  # orden opcional de categorías en Y
    colorscale="YlGnBu",  # paleta secuencial (puedes usar "Viridis", "Blues", etc.)
    decimals=2,           # decimales para el texto mostrado
    title="Rendimiento por Dado × Pellet"
):
    # 1) Tabla dinámica (pivot)
    pt = pd.pivot_table(
        df, index=y_col, columns=x_col, values=value_col, aggfunc=aggfunc
    )

    # 2) Orden opcional de categorías
    if x_order is None:
        x_order = list(pt.columns)
    if y_order is None:
        y_order = list(pt.index)

    pt = pt.reindex(index=y_order, columns=x_order)

    # 3) Matrices para el heatmap y el texto
    z = pt.values
    text = np.where(np.isnan(z), "", np.vectorize(lambda v: f"{v:.{decimals}f}")(z))

    # 4) Figura Heatmap con bordes entre celdas (xgap/ygap)
    fig = go.Figure(
        data=go.Heatmap(
            z=z,
            x=x_order,
            y=y_order,
            colorscale=colorscale,
            zmin=np.nanmin(z) if np.isfinite(z).any() else None,
            zmax=np.nanmax(z) if np.isfinite(z).any() else None,
            colorbar=dict(title=value_col),
            hovertemplate=f"{x_col}=%{{x}}<br>{y_col}=%{{y}}<br>{value_col}=%{{z:.{decimals}f}}<extra></extra>",
            xgap=1,  # crea “cuadros” separados (bordes visibles)
            ygap=1,
        )
    )

    # 5) Anotaciones de texto en cada celda
    annotations = []
    for i, yy in enumerate(y_order):
        for j, xx in enumerate(x_order):
            if text[i][j] != "":
                annotations.append(dict(
                    x=xx, y=yy, text=text[i][j],
                    showarrow=False,
                    font=dict(color="black", size=11)
                ))
    fig.update_layout(annotations=annotations)

    # 6) Estilo pro: marcos y fondo transparente
    axis_common = dict(
        showline=True, linecolor="black", linewidth=1.5, mirror=True,
        ticks="outside", ticklen=6,
        gridcolor="rgba(0,0,0,0.12)"
    )
    fig.update_xaxes(**axis_common)
    fig.update_yaxes(**axis_common, autorange="reversed")  # convención matricial

    fig.update_layout(
        title=dict(text=title, x=0.5),
        margin=dict(l=80, r=60, t=80, b=80),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        font=dict(family="Arial, Helvetica, sans-serif", size=12)
    )
    return fig


In [31]:
def cat_heatmap_duallabels(
    df: pd.DataFrame,
    x_col: str,                # eje X (categórico) p.ej. "Dado"
    y_col: str,                # eje Y (categórico) p.ej. "pellet"
    value_col: str,            # métrica para colorear + etiqueta central (p.ej. "rendimiento")
    side_col: str,             # métrica secundaria para etiqueta lateral (p.ej. "toneladas_procesadas")
    aggfunc="mean",            # agregación para ambas métricas: "mean" | "sum" | "median" | ...
    x_order: list | None = None,
    y_order: list | None = None,
    colorscale="YlGnBu",
    decimals_value=2,          # decimales para value_col (centro)
    decimals_side=1,           # decimales para side_col (lateral)
    title="Rendimiento por Dado × pellet",
    side_position="bottom",     # "right" | "left" | "top" | "bottom"
    side_prefix="",            # prefijo para la etiqueta lateral, p.ej. "t=" o "Tn="
    center_font_size=12,
    side_font_size=10
):
    """
    Crea un heatmap categórico con:
      - color por 'value_col'
      - etiqueta central = value_col
      - etiqueta lateral = side_col (en el borde indicado por side_position)
    """

    # 1) Tablas dinámicas para ambas métricas
    pt_val = pd.pivot_table(df, index=y_col, columns=x_col, values=value_col, aggfunc=aggfunc)
    pt_side = pd.pivot_table(df, index=y_col, columns=x_col, values=side_col, aggfunc=aggfunc)

    # 2) Orden opcional de categorías
    if x_order is None: x_order = list(pt_val.columns)
    if y_order is None: y_order = list(pt_val.index)

    pt_val = pt_val.reindex(index=y_order, columns=x_order)
    pt_side = pt_side.reindex(index=y_order, columns=x_order)

    # 3) Matriz para el heatmap y textos
    z = pt_val.values.astype(float)
    # Textos centrales (value_col)
    text_center = np.where(
        np.isnan(z), "",
        np.vectorize(lambda v: f"{v:.{decimals_value}f}")(z)
    )
    # Textos laterales (side_col)
    z_side = pt_side.values.astype(float)
    text_side = np.where(
        np.isnan(z_side), "",
        np.vectorize(lambda v: f"{side_prefix}{v:.{decimals_side}f}")(z_side)
    )

    # 4) Figura base Heatmap (con gaps para “cuadros”)
    fig = go.Figure(go.Heatmap(
        z=z,
        x=x_order,
        y=y_order,
        colorscale=colorscale,
        zmin=np.nanmin(z) if np.isfinite(z).any() else None,
        zmax=np.nanmax(z) if np.isfinite(z).any() else None,
        colorbar=dict(title=value_col),
        hovertemplate=f"{x_col}=%{{x}}<br>{y_col}=%{{y}}"
                      f"<br>{value_col}=%{{z:.{decimals_value}f}}",
        # Pasamos side_col al hover con customdata
        customdata=np.where(np.isnan(z_side), "", np.vectorize(lambda v: f"{v:.{decimals_side}f}")(z_side)),
        xgap=1, ygap=1
    ))

    # 5) Anotaciones: centro y lateral
    annotations = []
    for i, yy in enumerate(y_order):
        for j, xx in enumerate(x_order):
            center_txt = text_center[i][j]
            side_txt = text_side[i][j]
            if center_txt != "":
                annotations.append(dict(
                    x=xx, y=yy, text=center_txt,
                    showarrow=False,
                    font=dict(color="black", size=center_font_size)
                ))
            if side_txt != "":
                # Colocación lateral con xshift/yshift (en pixels)
                xshift = yshift = 0
                xanchor = "center"; yanchor = "middle"
                if side_position == "right":
                    xshift = 18; xanchor = "left";  yanchor = "middle"
                elif side_position == "left":
                    xshift = -18; xanchor = "right"; yanchor = "middle"
                elif side_position == "top":
                    yshift = 14; xanchor = "center"; yanchor = "bottom"
                elif side_position == "bottom":
                    yshift = -14; xanchor = "center"; yanchor = "top"

                annotations.append(dict(
                    x=xx, y=yy, text=side_txt,
                    showarrow=False,
                    xshift=xshift, yshift=yshift,
                    xanchor=xanchor, yanchor=yanchor,
                    font=dict(color="black", size=side_font_size)
                ))

    fig.update_layout(annotations=annotations)

    # 6) Estilo pro: marcos y fondo transparente
    axis_common = dict(
        showline=True, linecolor="black", linewidth=1.5, mirror=True,
        ticks="outside", ticklen=6,
        gridcolor="rgba(0,0,0,0.12)"
    )
    fig.update_xaxes(**axis_common)
    fig.update_yaxes(**axis_common, autorange="reversed")

    fig.update_layout(
        title=dict(text=title, x=0.5),
        margin=dict(l=80, r=60, t=80, b=80),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        font=dict(family="Arial, Helvetica, sans-serif", size=12)
    )
    return fig


In [32]:
summary_dado_pellet = summary(
    df=df_prod_pel, cols_group=["Dado", "pellet"])
summary_dado_pellet

,Dado,pellet,toneladas_procesadas,horas_trabajadas,rendimiento_medio,pdi_medio,carga_media,bultos_procesado,rendimiento_total
0,0.0,pellet 350,16.00,6.500000,2.666667,96.5,17.0,400.0,2.461538
1,1.0,pellet 350,2699.48,941.333333,2.838182,96.0,16.0,67487.0,2.867720
2,1.0,pellet 420,660.28,137.583333,4.800000,97.2,28.0,16507.0,4.799128
3,1.0,pellet 520,4919.36,659.500000,7.346939,96.0,16.0,122984.0,7.459227
4,2.0,pellet 350,127.56,47.833333,2.666667,96.8,16.0,3189.0,2.666760
5,2.0,pellet 420,858.00,151.333333,5.647059,95.6,28.0,21450.0,5.669604
6,3.0,pellet 350,20.00,7.416667,2.640000,96.2,15.0,500.0,2.696629
7,3.0,pellet 420,214.00,43.750000,4.800000,97.0,26.0,5350.0,4.891429
8,3.0,pellet 520,10.00,1.500000,6.000000,96.0,16.0,250.0,6.666667
9,4.0,pellet 420,2692.40,493.916667,5.600000,96.8,27.0,67310.0,5.451122


In [33]:
# df contiene columnas: "Dado", "pellet", "rendimiento", "toneladas_procesadas"
fig = cat_heatmap_duallabels(
    summary_dado_pellet,
    x_col="Dado",
    y_col="pellet",
    value_col="rendimiento_medio",
    side_col="toneladas_procesadas",
    aggfunc="mean",                 # promedio por intersección
    colorscale="YlGnBu",
    decimals_value=2,
    decimals_side=1,
    title="Rendimiento (centro) y Ton procesadas (lateral) — Dado × Pellet",
    side_position="bottom",          # 'right'|'left'|'top'|'bottom'
    side_prefix="Tn=",              # prefijo en la etiqueta lateral
)
fig.show()
# fig.write_html("dado_pellet_heatmap.html", include_plotlyjs="cdn")


In [34]:
# Supón que tu DataFrame se llama df y tiene columnas: "Dado", "pellet" y "rendimiento"
fig = cat_heatmap(
    summary_dado_pellet,
    x_col="Dado",
    y_col="pellet",
    value_col="rendimiento_medio",
    aggfunc="mean",                # promedio por intersección
    colorscale="YlGnBu",
    decimals=2,
    title="Rendimiento medio por Dado × pellet"
)
fig.show()
# o: fig.write_html("rendimiento_heatmap.html", include_plotlyjs="cdn")


In [35]:
summary_pelletizador_pellet = summary(
    df=df_prod_pel, cols_group=["Pelletizador", "pellet"])
summary_pelletizador_pellet

,Pelletizador,pellet,toneladas_procesadas,horas_trabajadas,rendimiento_medio,pdi_medio,carga_media,bultos_procesado,rendimiento_total
0,Felipe,pellet 350,319.60,121.416667,2.594595,96.000,16.0,7990.0,2.632258
1,Felipe,pellet 420,844.24,157.833333,5.285161,96.200,25.0,21106.0,5.348933
2,Felipe,pellet 520,1331.16,184.000000,7.200000,96.200,16.0,33279.0,7.234565
3,Felipe-Jorge,pellet 420,67.00,9.333333,6.257938,96.200,25.0,1675.0,7.178571
4,Felipe-Jorge,pellet 520,55.00,7.500000,7.350000,98.000,16.0,1375.0,7.333333
...,...,...,...,...,...,...,...,...,...
66,Yohan-Luis,pellet 520,157.00,21.750000,6.857143,95.800,16.5,3925.0,7.218391
67,Yohan-WIlson,pellet 420,466.48,82.083333,6.000000,96.800,27.0,11662.0,5.683005
68,Yohan-Wilson,pellet 350,67.00,25.333333,2.880000,96.205,16.0,1675.0,2.644737
69,Yohan-Wilson,pellet 520,777.00,104.833333,7.741935,96.800,16.0,19425.0,7.411765


In [36]:
fig = cat_heatmap(
    summary_pelletizador_pellet,
    y_col="Pelletizador",
    x_col="pellet",
    value_col="rendimiento_medio",
    aggfunc="mean",                # promedio por intersección
    colorscale="YlGnBu",
    decimals=2,
    title="Rendimiento medio por Pelletizador × pellet"
)
fig.show()